In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
warnings.simplefilter(action='ignore')

In [2]:
data_train_final = pd.read_csv('data_train_final.csv')
data_test_final = pd.read_csv('data_test_final.csv')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense
import keras_nlp

In [5]:
# Prepare baskets
def prepare_baskets(data):
    return data.groupby("order_id")["product_id"].apply(list).tolist()

train_baskets = prepare_baskets(data_train_final)
test_baskets = prepare_baskets(data_test_final)

train_baskets, val_baskets = train_test_split(train_baskets, test_size=0.25, random_state=42)

max_len = max(len(basket) for basket in train_baskets + val_baskets + test_baskets)

In [6]:
D = 32
batch_size = 256
max_epochs = 1000
lr = 1e-4
max_items = len(set(data_train_final['product_id']))

In [7]:
def preprocess_baskets(baskets):
    context_inputs = []
    target_inputs = []
    masked_idxs = []

    for basket in baskets:
        for idx, elt in enumerate(basket):
            target_inputs.append(elt)
            context_inputs.append(basket[:idx] + [max_items + 1] + basket[(idx+1):])
            masked_idxs.append(idx)

    context_inputs = pad_sequences(context_inputs, padding='post', maxlen = max_len, value=0)
    return np.array(context_inputs), np.array(target_inputs) - 1, np.array(masked_idxs)

train_context_input, train_target_input, train_masked_idxs = preprocess_baskets(train_baskets)
val_context_input, val_target_input, val_masked_idxs = preprocess_baskets(val_baskets)
test_context_input, test_target_input, test_masked_idxs = preprocess_baskets(test_baskets)

In [8]:
train_context_input[2]

array([55, 54, 64, 20,  4,  1, 29, 21, 11,  8, 23,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [9]:
train_target_input[2]

11

In [10]:
input_context = layers.Input(shape=(max_len,), dtype=tf.int32, name="context_input")
masked_idx_input = layers.Input(shape=(1,), dtype=tf.int32, name="masked_idx_input")

alpha_embedding = layers.Embedding(input_dim=max_items + 2, output_dim=D, name="alpha_embedding")
context_embedding = alpha_embedding(input_context) 

class ZeroMaskEmbedding(layers.Layer):
    def call(self, embeddings, input_tokens):
        mask = tf.cast(tf.not_equal(input_tokens, 0), tf.float32) 
        mask = tf.expand_dims(mask, axis=-1) 
        return embeddings * mask 

context_embedding = ZeroMaskEmbedding()(context_embedding, input_context)

class MaskLayer(layers.Layer):
    def call(self, input_context, position):
        return position * tf.expand_dims(tf.cast(tf.not_equal(input_context, 0), tf.float32), axis = -1)

position = keras_nlp.layers.PositionEmbedding(sequence_length=max_len)(context_embedding)
masked_position = MaskLayer()(input_context, position)
context_embedding = context_embedding + masked_position

attention_layer_1 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_1")
attn_output_1 = attention_layer_1(
    query=context_embedding,
    value=context_embedding,
    key=context_embedding,
    use_causal_mask=True
)

attn_output_1 = context_embedding + attn_output_1

attention_layer_2 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_2")
attn_output_2 = attention_layer_2(
    query=attn_output_1,
    value=attn_output_1,
    key=attn_output_1,
    use_causal_mask=True
)

context_embedding = attn_output_1 + attn_output_2

class GatherLayer(layers.Layer):
    def call(self, inputs):
        context_embedding, masked_idx_input = inputs
        return tf.gather(context_embedding, indices=tf.squeeze(masked_idx_input, axis=-1), batch_dims=1)

masked_embeddings = GatherLayer()([context_embedding, masked_idx_input])

output = layers.Dense(max_items, activation="softmax", name="output_layer", use_bias = False)(masked_embeddings)

model = Model(inputs=[input_context, masked_idx_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="sparse_categorical_crossentropy")

early_stopping = EarlyStopping(monitor='val_loss', patience=1000, restore_best_weights=True)

history = model.fit(
    [train_context_input, train_masked_idxs], train_target_input,
    validation_data=([val_context_input, val_masked_idxs], val_target_input),
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping]
)

Epoch 1/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - loss: 4.0943 - val_loss: 3.9739
Epoch 2/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9688 - val_loss: 3.9499
Epoch 3/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9491 - val_loss: 3.9388
Epoch 4/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9396 - val_loss: 3.9292
Epoch 5/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9276 - val_loss: 3.9161
Epoch 6/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9159 - val_loss: 3.9022
Epoch 7/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.9001 - val_loss: 3.8918
Epoch 8/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.8908 - val_loss: 3.8838
Epoch 9/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.8828 - val_loss: 3.8785
Epoch 10/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.8781 - val_loss: 3.8745
Epoch 11/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.8755 - val_loss: 3.8715
Epoch 12/1000
807/

Epoch 133/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7527 - val_loss: 3.7623
Epoch 134/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7511 - val_loss: 3.7619
Epoch 135/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7538 - val_loss: 3.7617
Epoch 136/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7564 - val_loss: 3.7614
Epoch 137/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7515 - val_loss: 3.7613
Epoch 138/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7506 - val_loss: 3.7610
Epoch 139/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7512 - val_loss: 3.7608
Epoch 140/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7483 - val_loss: 3.7606
Epoch 141/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7509 - val_loss: 3.7607
Epoch 142/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7523 - val_loss: 3.7600
Epoch 143/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7525 - val_loss: 3.7598

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7239 - val_loss: 3.7384
Epoch 264/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7216 - val_loss: 3.7382
Epoch 265/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7253 - val_loss: 3.7380
Epoch 266/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7209 - val_loss: 3.7374
Epoch 267/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7236 - val_loss: 3.7380
Epoch 268/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7228 - val_loss: 3.7375
Epoch 269/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7235 - val_loss: 3.7375
Epoch 270/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.7236 - val_loss: 3.7375
Epoch 271/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7195 - val_loss: 3.7370
Epoch 272/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7230 - val_loss: 3.7370
Epoch 273/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7236 - val_loss: 3.7370
Epoch 274/1000

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7069 - val_loss: 3.7263
Epoch 394/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7074 - val_loss: 3.7262
Epoch 395/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7062 - val_loss: 3.7259
Epoch 396/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7068 - val_loss: 3.7261
Epoch 397/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7053 - val_loss: 3.7258
Epoch 398/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7035 - val_loss: 3.7259
Epoch 399/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7027 - val_loss: 3.7259
Epoch 400/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7041 - val_loss: 3.7255
Epoch 401/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7040 - val_loss: 3.7260
Epoch 402/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7059 - val_loss: 3.7256
Epoch 403/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.7050 - val_loss: 3.7257
Epoch 404/1000

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6935 - val_loss: 3.7210
Epoch 524/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6938 - val_loss: 3.7208
Epoch 525/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6929 - val_loss: 3.7211
Epoch 526/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6954 - val_loss: 3.7210
Epoch 527/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6960 - val_loss: 3.7210
Epoch 528/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6911 - val_loss: 3.7208
Epoch 529/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6990 - val_loss: 3.7204
Epoch 530/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6927 - val_loss: 3.7207
Epoch 531/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6953 - val_loss: 3.7210
Epoch 532/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6944 - val_loss: 3.7205
Epoch 533/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6928 - val_loss: 3.7204
Epoch 534/1000

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6838 - val_loss: 3.7187
Epoch 654/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6825 - val_loss: 3.7188
Epoch 655/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6855 - val_loss: 3.7187
Epoch 656/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6838 - val_loss: 3.7188
Epoch 657/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6857 - val_loss: 3.7185
Epoch 658/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6835 - val_loss: 3.7189
Epoch 659/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6853 - val_loss: 3.7193
Epoch 660/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6871 - val_loss: 3.7186
Epoch 661/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6822 - val_loss: 3.7187
Epoch 662/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6833 - val_loss: 3.7188
Epoch 663/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6836 - val_loss: 3.7187
Epoch 664/1000

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6781 - val_loss: 3.7187
Epoch 784/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6821 - val_loss: 3.7186
Epoch 785/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6807 - val_loss: 3.7189
Epoch 786/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6808 - val_loss: 3.7187
Epoch 787/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6783 - val_loss: 3.7187
Epoch 788/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6791 - val_loss: 3.7191
Epoch 789/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6817 - val_loss: 3.7194
Epoch 790/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6761 - val_loss: 3.7192
Epoch 791/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6794 - val_loss: 3.7190
Epoch 792/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6777 - val_loss: 3.7191
Epoch 793/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6814 - val_loss: 3.7190
Epoch 794/1000

807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6712 - val_loss: 3.7194
Epoch 914/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6716 - val_loss: 3.7197
Epoch 915/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.6745 - val_loss: 3.7193
Epoch 916/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.6718 - val_loss: 3.7193
Epoch 917/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6757 - val_loss: 3.7195
Epoch 918/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.6737 - val_loss: 3.7197
Epoch 919/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.6755 - val_loss: 3.7195
Epoch 920/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6742 - val_loss: 3.7199
Epoch 921/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6731 - val_loss: 3.7194
Epoch 922/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - loss: 3.6694 - val_loss: 3.7196
Epoch 923/1000
807/807 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 3.6772 - val_loss: 3.7195
Epoch 924/1000

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context_input       │ (None, 24)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha_embedding     │ (None, 24, 32)    │      2,080 │ context_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_mask_embedding │ (None, 24, 32)    │          0 │ alpha_embedding[… │
│ (ZeroMaskEmbedding) │                   │            │ context_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, 24, 32)    │        768 │ zero_mask_embedd… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_layer          │ (None, 24, 32)    │          0 │ context_input[0]… │
│ (MaskLayer)         │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 24, 32)    │          0 │ zero_mask_embedd… │
│                     │                   │            │ mask_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 32)    │      4,224 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0],        │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 24, 32)    │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 24, 32)    │      4,224 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0],      │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 24, 32)    │          0 │ add_1[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masked_idx_input    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gather_layer        │ (None, 32)        │          0 │ add_2[0][0],      │
│ (GatherLayer)       │                   │            │ masked_idx_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 63)        │      2,016 │ gather_layer[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,938 (156.01 KB)

 Trainable params: 13,312 (52.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,626 (104.01 KB)

In [12]:
# Evaluate on Test Data
test_loss = model.evaluate([test_context_input, test_masked_idxs], test_target_input, batch_size=batch_size)
print(f"Test Loss: {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.6978
Test Loss: 3.7064034938812256


In [13]:
alpha_embedding_layer = model.get_layer("alpha_embedding")
alpha_embedding_weights = alpha_embedding_layer.get_weights()[0][1:-1]

In [14]:
output_layer = model.get_layer("output_layer")
output_layer_weights = output_layer.get_weights()[0]

In [15]:
sim_matrix = pd.DataFrame(np.matmul(alpha_embedding_weights, output_layer_weights) + \
    np.matmul(output_layer_weights.T, alpha_embedding_weights.T))
sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,-1.841904,1.108828,-0.730162,0.717771,-0.199490,-0.180262,-0.435006,-0.958158,1.089629,0.349002,...,-0.432976,0.482929,0.089556,0.092750,0.051487,-0.323619,-0.328753,0.610504,0.384999,0.884012
1,1.108828,0.007831,0.396000,-0.125406,0.080645,-0.711622,0.767906,0.303356,0.014885,-0.500323,...,0.566183,0.278292,-0.334465,0.115957,0.317823,-0.046111,-0.984802,0.646771,0.551754,0.882254
2,-0.730162,0.396000,0.499883,-0.045246,0.145828,-0.016850,0.806530,1.035866,-0.230337,0.444847,...,-0.332298,0.481008,-0.990566,-0.048352,0.055188,-0.753206,-1.258053,-0.177289,-0.151948,-0.294468
3,0.717771,-0.125406,-0.045246,0.428570,-0.934990,-0.019234,-2.432556,0.077797,-0.469771,0.092044,...,1.146295,-0.322390,-0.404144,-0.767939,0.373942,0.363779,1.063499,0.064191,-0.944657,0.122299
4,-0.199490,0.080645,0.145828,-0.934990,0.679590,0.461047,0.693194,0.254980,-0.347969,-0.110327,...,-0.467710,-0.110833,0.329302,0.225049,0.090979,-0.336216,-0.249298,-0.193985,-0.476760,-0.253856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-0.323619,-0.046111,-0.753206,0.363779,-0.336216,0.093432,-0.912742,-0.441715,-0.514504,0.340376,...,-0.477930,0.254043,0.124370,0.010017,0.298697,-0.013870,-0.590100,-0.305164,0.631588,0.532054
59,-0.328753,-0.984802,-1.258053,1.063499,-0.249298,-1.176764,1.056181,0.288590,0.348372,0.481163,...,-0.354965,0.228785,0.559258,-0.168537,0.195038,-0.590100,0.231886,0.212517,0.498121,0.290149
60,0.610504,0.646771,-0.177289,0.064191,-0.193985,-1.350728,-0.725547,-0.372041,-0.209719,0.417564,...,0.048400,-0.294603,-0.288244,-0.318247,0.615774,-0.305164,0.212517,1.190751,0.054151,0.355407
61,0.384999,0.551754,-0.151948,-0.944657,-0.476760,0.620597,0.016610,-0.509385,-0.478836,0.080950,...,-0.230778,-0.685438,0.442669,0.383077,0.395985,0.631588,0.498121,0.054151,-0.204970,-0.155758


In [23]:
for i in range(sim_matrix.shape[0]):
    sim_matrix[i][i] = -1000

In [24]:
top_5_indices_desc = np.argsort(sim_matrix, axis=1)[:, -5:][:, ::-1]

top_5_dict = {}
for i in range(top_5_indices_desc.shape[0]):
    top_5_dict[i] = list(top_5_indices_desc[i])

In [25]:
products = pd.read_csv('products.csv')
products_dict = {}
for i in range(products.shape[0]):
    products_dict[products['product_id'][i]] = products['product_name'][i]

In [26]:
product_ids = [21903, 30391, 46667, 13176, 21616,  8518, 22935,  5876, 48679,
       24838, 31717, 47209, 26209, 34969, 27966, 37646, 44632, 16797,
       39275,  5077, 10749, 49235, 21137, 28204, 21938, 46979, 47626,
       44359, 34126, 28985, 24852, 41950, 30489,  9076, 24964, 45007,
       42265, 49683, 47766, 39877, 19057, 40706,  5450, 43961, 39928,
       22825, 12341, 17794,  4605, 22035, 27845, 27104, 26604,  8277,
        4920, 25890, 31506, 35951, 45066, 24184, 19660, 27086, 43352]

all_products = []
for i in product_ids:
    all_products.append(products_dict[i])

In [27]:
top_5_dict_items = {}

for k, v in top_5_dict.items():
    key = all_products[k]
    value = [all_products[val] for val in v]
    
    top_5_dict_items[key] = value

In [20]:
top_5_dict_items

{'Organic Baby Spinach': ['Organic Red Bell Pepper',
  'Organic Whole String Cheese',
  'Organic Tomato Cluster',
  'Organic Cucumber',
  'Organic Garnet Sweet Potato (Yam)'],
 'Organic Cucumber': ['Organic Raspberries',
  'Organic Large Extra Fancy Fuji Apple',
  'Organic Hass Avocado',
  'Organic Baby Spinach',
  'Organic Zucchini'],
 'Organic Ginger Root': ['100% Whole Wheat Bread',
  'Organic Zucchini',
  'Organic Lemon',
  'Organic Blueberries',
  'Organic Raspberries'],
 'Bag of Organic Bananas': ['Organic Hass Avocado',
  'Organic Strawberries',
  'Organic Whole Milk',
  'Strawberries',
  'Organic Avocado'],
 'Organic Baby Arugula': ['Yellow Onions',
  'Organic Whole Milk',
  'Organic Yellow Onion',
  'Organic Zucchini',
  'Organic Baby Arugula'],
 'Organic Red Onion': ['Organic Raspberries',
  'Large Lemon',
  'Carrots',
  'Seedless Red Grapes',
  'Organic Blackberries'],
 'Organic Yellow Onion': ['Organic Zucchini',
  'Organic Blackberries',
  'Red Peppers',
  'Fresh Cauliflow